In [9]:
class LudoEnv:
    def __init__(self):
        self.goal = 10
        self.num_players = 2
        self.pieces_per_player = 4
        self.state = [[0] * self.pieces_per_player for _ in range(self.num_players)] # 2D list representing player`s pieces positions
        self.current_player = 0  # The current player who is taking the turn (0 or 1)
        self.done = False  # Game over flag 
        self.safe_spots = [5, 15, 25, 35, 45]  # List of safe spots where players can rest their pieces (not affected by other players)

    def reset(self):
        self.state = [[0] * self.pieces_per_player for _ in range(self.num_players)] # Reset the state for a new game, all pieces are at the starting position
        self.current_player = 0
        self.done = False
        return self.get_state()

    def get_state(self):
        # Return the current state of the game(including each player's pieces positions & current player)
        return tuple(tuple(player) for player in self.state), self.current_player
    
    def get_valid_actions(self):
                # ترجع القطع التي يمكن تحريكها (أي التي لم تصل للهدف بعد)
        return [i for i, pos in enumerate(self.state[self.current_player]) if pos < self.goal]

    def move_piece(self, player, piece_index, steps):
                # تحريك قطعة اللاعب بمقدار خطوات معينة
        piece_position = self.state[player][piece_index] #الموضع الحالي للقطعة
        new_position = piece_position + steps # الموضع الجديد بعد التحريك
        
        # تحقق من الأكل
        opponent = 1 if player == 0 else 0 # تحديد اللاعب المنافس 
                # التحقق مما إذا تم ضرب قطعة للخصم في مكان غير آمن
        for i in range(self.pieces_per_player):
            if self.state[opponent][i] == new_position:
                self.state[opponent][i] = 0  # إرجاع قطعة الخصم للبداية
                return 20, -5  # زيادة المكافأة + عقوبة أقل للخصم

        # تحقق من الخانات الآمنة
        if new_position in self.safe_spots:
            return 5, 0  # مكافأة صغيرة للوصول إلى خانة آمنة

        if new_position == self.goal:  # إذا وصلت إلى الهدف
            return 20, 0  # زيادة المكافأة

        return 0, 0

    def step(self, action):
        dice_roll = random.randint(1, 6)
        reward = 0
        player_pieces = self.state[self.current_player]
        
        # تحريك القطعة
        if player_pieces[action] < self.goal:
                        # إذا كانت القطعة لم تصل للنهاية فقم بتحريكها
            reward, penalty = self.move_piece(self.current_player, action, dice_roll)
            player_pieces[action] = min(self.goal, player_pieces[action] + dice_roll)

        # إذا تم أكل قطعة، إضافة العقوبة
        reward -= penalty

        # إذا كانت جميع القطع وصلت
        if all(p == self.goal for p in player_pieces):#التحقق إذا فاز اللاعب
            reward += 50
            self.done = True

        # تغيير الدور
        self.current_player = 1 - self.current_player #تبديل الدور للاعب الآخر

        return self.get_state(), reward, self.done
        


In [11]:
class QLearningAgent:
    def __init__(self, actions, alpha=0.2, gamma=0.95, epsilon=0.1):

        #جدول Q لتخزين (state, action) -> Q-value
        self.q_table = {}
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

        #(مثلاً: [0,1,2,3])
        self.actions = actions

    def get_q_value(self, state, action):
        return self.q_table.get((state, action), 0.0)

    def choose_action(self, state, valid_actions):
        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        q_vals = [self.get_q_value(state, a) for a in valid_actions]
        max_q = max(q_vals)
        best_actions = [a for a, q in zip(valid_actions, q_vals) if q == max_q]
        return random.choice(best_actions)

    def learn(self, state, action, reward, next_state, next_valid_actions):
        current_q = self.get_q_value(state, action)
        max_next_q = max([self.get_q_value(next_state, a) for a in next_valid_actions], default=0)
        
         # نحسب القيمة الجديدة باستخدام معادلة Q-Learning
        new_q = current_q + self.alpha * (reward + self.gamma * max_next_q - current_q)
        # update q-table
        self.q_table[(state, action)] = new_q
